In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
#read origin img and change it to binary
image = cv2.imread('lena.bmp',0)
rows,cols = image.shape
threshold,bin_img = cv2.threshold(image,127,255,cv2.THRESH_BINARY)
cv2.imwrite("binary.bmp",bin_img)
bin_array = np.asarray(bin_img)

In [3]:
#initialize kernel
kernel = np.array([[0, 1, 1, 1, 0],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [0, 1, 1, 1, 0]])

### Dilation

In [60]:
def color(pic,r,c,i,j,k,k_r, k_c,s_r,s_c):
    r,c = pic.shape 
    for a in range(k_r):
        for b in range(k_c):
            if k[a][b] == 1:
                if i+a-s_r < 0 or j+b-s_c < 0 or i+a-s_r > r-1 or j+b-s_c > c-1:
                    continue
                pic[i+a-s_r][j+b-s_r] = 255
            
def Dilation(pic,out,k):
    r,c = pic.shape
    k_r, k_c = k.shape
    s_r = k_r//2
    s_c = k_c//2
    for i in range(r):
        for j in range(c):
            if pic[i][j] == 255:
                color(out,r,c,i,j,k,k_r,k_c,s_r,s_c)

dilation = np.full((rows,cols),0)        
Dilation(bin_array,dilation,kernel)
cv2.imwrite("dilation.bmp",dilation)

True

### Erosion

In [66]:
def check(pic,r,c,i,j,k,k_r,k_c,s_r,s_c):
    for a in range(k_c):
        for b in range(k_r):
            if k[a][b] == 1 :
                if i+a-s_r < 0 or j+b-s_c < 0 or i+a-s_r > r-1 or j+b-s_c > c-1:
                    continue
                if pic[i+a-s_r][j+b-s_r] != 255:
                    return -1
    return 1


def Erosion(pic,out,k): 
    r,c = pic.shape
    k_r, k_c = k.shape
    s_r = k_r//2
    s_c = k_c//2
    for i in range(r):
        for j in range(c):
            if check(pic,r,c,i,j,k,k_r, k_c,s_r,s_c) == 1:
                out[i][j] = 255            

erosion = np.full((rows,cols),0)
Erosion(bin_array,erosion,kernel)
cv2.imwrite("erosioin.bmp",erosion)

True

### Opening

In [67]:
# Erosion first and then Dilation 
opening = np.full((rows,cols),0)
Dilation(erosion,opening,kernel)
cv2.imwrite("open.bmp",opening)

True

### Closing

In [68]:
# Dilation first and then Erosion 
closing = np.full((rows,cols),0)
Erosion(dilation,closing,kernel)
cv2.imwrite("close.bmp",closing)

True

### Hit-and-miss transform

In [69]:
J = np.array([[0, 0, 0],
              [1, 1, 0],
              [0, 1, 0]])

K = np.array([[0, 1, 1],
              [0, 0, 1],
              [0, 0, 0]])

E_1 = np.full((rows,cols),0)
E_2 = np.full((rows,cols),0)
Hit = np.full((rows,cols),0)

bin_c = 255-bin_array
Erosion(bin_array,E_1,J)
Erosion(bin_c,E_2,K)
cv2.imwrite("E_1.bmp",E_1)
cv2.imwrite("E_2.bmp",E_2)

for i in range(rows):
    for j in range(cols):
        if E_1[i][j] == 255 and E_2[i][j] == 255:
            Hit[i][j] = 255
            
cv2.imwrite("hit.bmp",Hit)        

True